In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## functions

In [ ]:
import numpy as np
import time
from enum import Enum

from collections import Counter
from collections import defaultdict

SPEECH_PARTS = [
    'X',
    'ADJ',
    'ADV',
    'INTJ',
    'NOUN',
    'PROPN',
    'VERB',
    'ADP',
    'AUX',
    'CONJ',
    'SCONJ',
    'DET',
    'NUM',
    'PART',
    'PRON',
    'PUNCT',
    'GRND',
    'H',
    'R',
    'Q',
    'SYM',
]

SPEECH_PART_MAPPING = {str(s): num for num, s in enumerate(SPEECH_PARTS)}

MASK_VALUE = 0.0

MAX_LEN = 20

def build_speech_part_array(sp):
    output = [0. for _ in range(len(SPEECH_PARTS))]
    output[SPEECH_PART_MAPPING[str(sp)]] = 1.
    return output


CLASSES_NUM = [1, 3, 2, 4, 10, 9, 30, 86, 8, 7, 6, 17, 77, 37, 41, 90, 16, 34,
        28, 75, 64, 225, 12, 87, 14, 229, 43, 193, 38, 61, 35, 15, 53, 203, 68,
        236, 194, 230, 51, 39, 213, 231, 31, 33, 74, 209, 91, 96, 42, 204, 237,
        214, 88, 111, 18, 207, 69, 232, 5, 59, 97, 19, 60, 210, 89, 127, 169,
        274, 45, 106, 238, 47, 227, 240, 205, 208, 233, 57, 82, 46, 0]
# {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15, 16, 17, 18, 19, 28, 30, 31, 33, 34, 35, 37, 38, 39, 41, 42, 43, 45, 46, 47, 51, 53, 57, 59, 60, 61, 64, 68, 69, 74, 75, 77, 82, 86, 87, 88, 89, 90, 91, 96, 97, 106, 111, 127, 169, 193, 194, 203, 204, 205, 207, 208, 209, 210, 213, 214, 225, 227, 229, 230, 231, 232, 233, 236, 237, 238, 240, 274}
# set([i for i in range(313 + 1)]) 

CLASSES_NUM_MAPPING = {cn : num for num, cn in enumerate(CLASSES_NUM)}

def build_class_num_array(cn):
    output = [0. for _ in range(len(CLASSES_NUM))]
    output[CLASSES_NUM_MAPPING[cn]] = 1.
    return output


PARTS_MAPPING = {
    'UNKN': 0,
    'PREF': 1,
    'ROOT': 2,
    'SUFF': 3,
    'END': 4,
    'LINK': 5,
    'HYPH': 6,
    'POSTFIX': 7,
    'B-SUFF': 8,
    'B-PREF': 9,
    'B-ROOT': 10,
}

LETTERS = {
    'о': 1,
    'е': 2,
    'а': 3,
    'и': 4,
    'н': 5,
    'т': 6,
    'с': 7,
    'р': 8,
    'в': 9,
    'л': 10,
    'к': 11,
    'м': 12,
    'д': 13,
    'п': 14,
    'у': 15,
    'я': 16,
    'ы': 17,
    'ь': 18,
    'г': 19,
    'з': 20,
    'б': 21,
    'ч': 22,
    'й': 23,
    'х': 24,
    'ж': 25,
    'ш': 26,
    'ю': 27,
    'ц': 28,
    'щ': 29,
    'э': 30,
    'ф': 31,
    'ъ': 32,
    'ё': 33,
    '-': 34,
}

VOWELS = {
    'а', 'и', 'е', 'ё', 'о', 'у', 'ы', 'э', 'ю', 'я'
}


class MorphemeLabel(Enum):
    UNKN = 'UNKN'
    PREF = 'PREF'
    ROOT = 'ROOT'
    SUFF = 'SUFF'
    END = 'END'
    LINK = 'LINK'
    HYPH = 'HYPH'
    POSTFIX = 'POSTFIX'
    NONE = None


class Morpheme(object):
    def __init__(self, part_text, label, begin_pos):
        self.part_text = part_text
        self.length = len(part_text)
        self.begin_pos = begin_pos
        self.label = label
        self.end_pos = self.begin_pos + self.length

    def __len__(self):
        return self.length

    def get_labels(self):
        if self.length == 1:
            return ['S-' + self.label.value]
        result = ['B-' + self.label.value] 
        result += ['M-' + self.label.value for _ in self.part_text[1:-1]]
        result += ['E-' + self.label.value]
        return result

    def get_simple_labels(self):
        if (self.label == MorphemeLabel.SUFF or self.label == MorphemeLabel.PREF or self.label == MorphemeLabel.ROOT):

            result = ['B-' + self.label.value]
            if self.length > 1:
                result += [self.label.value for _ in self.part_text[1:]]
            return result
        else:
            return [self.label.value] * self.length

    def __str__(self):
        return self.part_text + ':' + self.label.value

    def __eq__(self, other):
        return self.part_text == other.part_text and self.label == other.label

    def __hash__(self):
        return hash(tuple([self.part_text, self.label]))

    @property
    def unlabeled(self):
        return not self.label.value


class Word(object):
    def __init__(self, morphemes=[], speech_part='X', class_num=0):
        self.morphemes = morphemes
        self.sp = speech_part
        self.class_num = class_num

    def append_morpheme(self, morpheme):
        self.morphemes.append(morpheme)

    def get_word(self):
        return ''.join([morpheme.part_text for morpheme in self.morphemes])

    def parts_count(self):
        return len(self.morphemes)

    def morpheme_count(self, morph_label):
        return len([morpheme for morpheme in self.morphemes
                    if morpheme.label == morph_label])

    def morpheme_indexes(self, morph_label):
        return [i for i, morpheme in enumerate(self.morphemes)
                    if morpheme.label == morph_label]

    def get_labels(self):
        result = []
        for morpheme in self.morphemes:
            result += morpheme.get_labels()
        return result

    def get_simple_labels(self):
        result = []
        for morpheme in self.morphemes:
            result += morpheme.get_simple_labels()
        return result

    def __str__(self):
        return '/'.join([str(morpheme) for morpheme in self.morphemes])

    def __len__(self):
        return sum(len(m) for m in self.morphemes)

    @property
    def unlabeled(self):
        return all(p.unlabeled for p in self.morphemes)


def parse_morpheme(str_repr, position):
    #print(str_repr)
    text, label = str_repr.split(':')
    return Morpheme(text, MorphemeLabel[label], position)


def parse_word(str_repr):
    if str_repr.count('\t') == 3:
        wordform, word_parts, _, class_info = str_repr.split('\t')
        if 'ADJ' in class_info:
            sp = 'ADJ'
        elif 'VERB' in class_info:
            sp = 'VERB'
        elif 'NOUN' in class_info:
            sp = 'NOUN'
        elif 'GRND' in class_info:
            sp = 'GRND'
        elif 'ADV' in class_info:
            sp = 'ADV'
        elif 'PART' in class_info:
            sp = 'PART'
        else:
            raise Exception("Unknown class", class_info)

        class_num = int( class_info [class_info.rfind(" ") + 1 : class_info.rfind(")")] )
        if class_num not in CLASSES_NUM:
            class_num = 1  if sp != 'ADV' else 0

    # таких случаев вообще не существует!
    elif str_repr.count('\t') == 2:
        wordform, word_parts, sp = str_repr.split('\t')
        class_num = 1
    else:
        wordform, word_parts = str_repr.split('\t')
        sp, class_num = 'X', 1

    if ':' in wordform or '/' in wordform:
        return None

    parts = word_parts.split('/')
    morphemes = []
    global_index = 0
    for part in parts:
        morphemes.append(parse_morpheme(part, global_index))
        global_index += len(part)
    #if Word(morphemes, sp, class_num).get_word() == 'еры':
    #    print(str_repr)
    return Word(morphemes, sp, class_num)


def divide_word(word):
    # Оставляем только слова-прилагательные с одним дефисом 
    if not (word.sp == 'ADJ' and word.morpheme_count(MorphemeLabel.HYPH) == 1):
        return []

    morphemes, new_words = [], []
    new_word_len = 0
    # Дробление слова по дефису
    """
    for i, morpheme in enumerate(word.morphemes):
        if morpheme.label == MorphemeLabel.HYPH:
            break
    """
    i = word.morpheme_indexes(MorphemeLabel.HYPH) [0]
    # Оставляем только слова, у которых перед дефисом стоит морфема о:LINK
    if word.morphemes[i-1] != Morpheme('о', MorphemeLabel.LINK, 0):
        return []

    # sp = 'H' # слово перед дефисом - что-то типо наречия(обозначим как 'H')
    # morphemes1 = word.morphemes[:i+1] # берем вместе с дефисом
    sp = 'ADV' # слово перед дефисом - наречие
    morphemes1 = word.morphemes[:i] # # берем первое слово до дефиса
    morphemes2 = word.morphemes[i+1:]
    if len(morphemes1) <= MAX_LEN and len(morphemes2) <= MAX_LEN:
        new_words.append(Word(morphemes1, sp, 0))
        new_words.append(Word(morphemes2, word.sp, word.class_num))


    """
    # Дробление слова по морфемам
    for morpheme in word.morphemes:
        if new_word_len + len(morpheme) > MAX_LEN:
            new_words.append(Word(morphemes, word.sp, word.class_num))
            morphemes = [morpheme]
            new_word_len = len(morpheme)
        else:
            morphemes.append(morpheme)
            new_word_len += len(morpheme)
    new_words.append(Word(morphemes, word.sp, word.class_num))

    print(f"\nDivide: {word}")
    print(*new_words, sep='\n')
    print()
    """
    return new_words

## main

In [ ]:
if __name__ == "__main__":

    train_set = "/content/drive/MyDrive/morph_model/tikhonov_lexemes.txt"

    # classes_num_dict = Counter()
    # sp_class_num = defaultdict(Counter)

    long_words = []

    train_part = []
    counter = 0
    if train_set:
        with open(train_set, 'r') as data:
            for num, line in enumerate(data):
                counter += 1
                word = parse_word(line.strip())  if line.strip() != '' else None
                if word is None:
                    continue
                # classes_num_dict [word.class_num] += 1
                # sp_class_num [word.sp] [word.class_num] += 1

                # train_part.append(word)

                if len(word) <= MAX_LEN:
                    train_part.append(word)
                else:
                    long_words.append(word)
                '''
                    for word in divide_word(word):
                        print(word.get_word())
                        #train_part.append(word)
                '''
                if counter % 10000 == 0:
                    print("Loaded", counter, "train words")
                #if counter == 1000: break
    
    #print(*train_part, sep='\n')

Loaded 20000 train words
Loaded 40000 train words
Loaded 50000 train words
Loaded 70000 train words
Loaded 80000 train words
Loaded 90000 train words
Loaded 100000 train words
Loaded 110000 train words
Loaded 120000 train words
Loaded 140000 train words
Loaded 150000 train words
Loaded 160000 train words
Loaded 170000 train words
Loaded 180000 train words
Loaded 190000 train words
Loaded 220000 train words
Loaded 230000 train words
Loaded 240000 train words
Loaded 250000 train words
Loaded 260000 train words


ValueError: ignored

## Статистика

### Статистика по 'class_num' в общем и его соответсвию конкретной части речи ('speech_part')

In [ ]:
    common_classes_num = classes_num_dict.most_common()
    print("\nСобранная статистика по class_num (first 20 common):")
    print(*common_classes_num[:20], sep='\n')
    for i in range(20, 100, 20):
        print("\nСобранная статистика по class_num (next 20 common):")
        print(*common_classes_num[i : i+20], sep='\n')
    print("\nСобранная статистика по class_num (others):")
    print(*common_classes_num[100:], sep='\n')

    common_classes_num = list(map(lambda x: x[0], common_classes_num[ : 150]))
    print("CLASSES_NUM:", common_classes_num[:60])
    print("CLASSES_NUM:", common_classes_num[:100])
    print("CLASSES_NUM:", common_classes_num[:150])

    print("\nspeech part and its class_num:")
    print(*sp_class_num.items(), sep='\n')

### Статистика по длиннным словам (> max_len)

In [ ]:
    long_words.sort(key=lambda word: len(word.get_word()), reverse=True)
    hyphen_words = list(filter(lambda word: word.morpheme_count(MorphemeLabel.HYPH) > 0, long_words))
    link_words = list(filter(lambda word: word.morpheme_count(MorphemeLabel.LINK) > 0 and
                            word.morpheme_count(MorphemeLabel.HYPH) == 0, long_words))
    
    simple_long_words = set(long_words) - set(hyphen_words) - set(link_words)
    simple_long_words = sorted(simple_long_words, key=lambda word: len(word.get_word()), reverse=True)

    root_count = list(map(lambda word: word.morpheme_count(MorphemeLabel.ROOT), long_words))

In [ ]:
def equal_lemmas(lexeme1, lexeme2): # равны без окончания
    return Word(lexeme1.morphemes[:-1]).get_word() == Word(lexeme2.morphemes[:-1]).get_word() 

def make_lemmas(lexemes):
    lemmas = [lexemes[0]]
    for i in range(1, len(lexemes)):
        if not equal_lemmas(lexemes[i], lexemes[i - 1]):
            lemmas.append(lexemes[i])
    return lemmas

In [ ]:
print(*long_words[0].morphemes[:-1], sep='\n', end='\n\n')
long_words[0].morphemes[:-1] == long_words[1].morphemes[:-1]

брон:ROOT
е:LINK
бой:ROOT
н:SUFF
о:LINK
-:HYPH
за:PREF
жиг:ROOT
а:SUFF
тельн:SUFF
о:LINK
-:HYPH
трасс:ROOT
ир:SUFF
у:SUFF
ющ:SUFF



True

In [ ]:
[d for d in dir(long_words[0].morphemes[0]) if not d.startswith("__")]

['begin_pos',
 'end_pos',
 'get_labels',
 'get_simple_labels',
 'label',
 'length',
 'part_text',
 'unlabeled']

In [ ]:
    output = open("/content/drive/MyDrive/morph_model/long_words.txt", 'w', encoding='utf-8')

    print(f"\nСобранная статистика по длинным словам (len>{MAX_LEN}):", file=output)
    print(f"Количество длинных слов: {len(long_words)} ({len(long_words) / (len(long_words) + len(train_part)) * 100 :.2f}% от общей выборки)", file=output)
    print(f"Среднее количество корней в длинных словах: {sum(root_count) / len(root_count) :.2f}", file=output)
    print(f"Количество длинных слов через дефис: {len(hyphen_words)} ({len(hyphen_words) / (len(long_words) + len(train_part)) * 100 :.2f}% от общей выборки, {len(hyphen_words) / len(long_words) * 100 :.2f}% от всех длинных слов)", file=output)
    print(f"Количество длинных слов через соединительную гласную: {len(link_words)} ({len(link_words) / (len(long_words) + len(train_part)) * 100 :.2f}% от общей выборки, {len(link_words) / len(long_words) * 100 :.2f}% от всех длинных слов)", file=output)
    print(f"Количество длинных слов без дефиса и без соединительной гласной: {len(simple_long_words)} ({len(simple_long_words) / (len(long_words) + len(train_part)) * 100 :.2f}% от общей выборки, {len(simple_long_words) / len(long_words) * 100 :.2f}% от всех длинных слов)", file=output)


    print("\nДлинные слова через дефис:", file=output)
    for word in make_lemmas(hyphen_words[:]):
        w = word.get_word()
        r = word.morpheme_count(MorphemeLabel.ROOT)
        print(f"(len={len(w)}) (roots={r}) {w}", file=output)

    print("\nДлинные слова через соединительную гласную:", file=output)
    for word in make_lemmas(link_words[:]):
        w = word.get_word()
        r = word.morpheme_count(MorphemeLabel.ROOT)
        print(f"(len={len(w)}) (roots={r}) {w}", file=output)

    print("\nДлинные слова без дефиса и без соединительной гласной:", file=output)
    for word in make_lemmas(simple_long_words[:]):
        w = word.get_word()
        r = word.morpheme_count(MorphemeLabel.ROOT)
        print(f"(len={len(w)}) (roots={r}) {w}", file=output)

    output.close()
    

In [ ]:
    long_words_21_22 = list(filter(lambda word: len(word) <= 22, long_words))
    hyphen_words_21_22 = list(filter(lambda word: len(word) <= 22, hyphen_words))
    link_words_21_22 = list(filter(lambda word: len(word) <= 22, link_words))
    simple_long_words_21_22 = list(filter(lambda word: len(word) <= 22, simple_long_words))

In [ ]:
    output = open("/content/drive/MyDrive/morph_model/long_words_21_22.txt", 'w', encoding='utf-8')
    
    print(f"\nСобранная статистика по словам длиной 21, 22 ({len(long_words_21_22) / len(long_words) * 100 :.2f}% от всех длинных слов):", file=output)
    print(f"Количество длинных слов через дефис: {len(hyphen_words_21_22)} ({len(hyphen_words_21_22) / (len(long_words) + len(train_part)) * 100 :.2f}% от общей выборки, \
{len(hyphen_words_21_22) / len(long_words) * 100 :.2f}% от всех длинных слов, \
{len(hyphen_words_21_22) / len(long_words_21_22) * 100 :.2f}% от всех слов длиной 21, 22, \
{len(hyphen_words_21_22) / len(hyphen_words) * 100 :.2f}% от всех слов через дефис)", file=output)
    print(f"Количество длинных слов через соединительную гласную: {len(link_words_21_22)} ({len(link_words_21_22) / (len(long_words) + len(train_part)) * 100 :.2f}% от общей выборки, \
{len(link_words_21_22) / len(long_words) * 100 :.2f}% от всех длинных слов, \
{len(link_words_21_22) / len(long_words_21_22) * 100 :.2f}% от всех слов длиной 21, 22, \
{len(link_words_21_22) / len(link_words) * 100 :.2f}% от всех слов через соед.гл.)", file=output)
    print(f"Количество длинных слов без дефиса и без соединительной гласной: {len(simple_long_words_21_22)} ({len(simple_long_words_21_22) / (len(long_words) + len(train_part)) * 100 :.2f}% от общей выборки, \
{len(simple_long_words_21_22) / len(long_words) * 100 :.2f}% от всех длинных слов, \
{len(simple_long_words_21_22) / len(long_words_21_22) * 100 :.2f}% от всех слов длиной 21, 22, \
{len(simple_long_words_21_22) / len(simple_long_words) * 100 :.2f}% от всех слов без дефиса и соед.гл.)", file=output)


    print("\nДлинные слова через дефис:", file=output)
    for word in make_lemmas(hyphen_words_21_22[:]):
        w = word.get_word()
        r = word.morpheme_count(MorphemeLabel.ROOT)
        print(f"(len={len(w)}) (roots={r}) {w}", file=output)

    print("\nДлинные слова через соединительную гласную:", file=output)
    for word in make_lemmas(link_words_21_22[:]):
        w = word.get_word()
        r = word.morpheme_count(MorphemeLabel.ROOT)
        print(f"(len={len(w)}) (roots={r}) {w}", file=output)

    print("\nДлинные слова без дефиса и без соединительной гласной:", file=output)
    for word in make_lemmas(simple_long_words_21_22[:]):
        w = word.get_word()
        r = word.morpheme_count(MorphemeLabel.ROOT)
        print(f"(len={len(w)}) (roots={r}) {w}", file=output)

    output.close()

In [ ]:
len(long_words_21_22) / (len(long_words) + len(train_part)) * 100

0.7032033164563711

### Статистика по морфемам, которые стоят перед дефисом в длинных словах и на конце наречий

In [ ]:
    hyphen_words = list(filter(lambda word: word.morpheme_count(MorphemeLabel.HYPH) == 1 and word.sp == 'ADJ', long_words))
    before_hyphen = []
    for word in hyphen_words:
        i = word.morpheme_indexes(MorphemeLabel.HYPH) [0] # позиция дефиса
        before_hyphen.append(word.morphemes[i-1])

    print("Всевозможные морфемы перед дефисом в длинных словах:")
    for morpheme, i in Counter(before_hyphen).most_common():
        print(f"({i}, {i/len(hyphen_words)*100:.2f}%) {morpheme}")


    adverbs = list(filter(lambda word: word.sp == 'ADV', train_part))
    end_adverbs = [word.morphemes[-1] for word in adverbs]

    print("Всевозможные морфемы на концах наречий:")
    for morpheme, i in Counter(end_adverbs).most_common():
        print(f"({i}, {i/len(adverbs)*100:.2f}%) {morpheme}")

Всевозможные морфемы перед дефисом в длинных словах:
(13098, 96.73%) о:LINK
(130, 0.96%) социал:ROOT
(53, 0.39%) ал:SUFF
(53, 0.39%) ый:END
(48, 0.35%) масс:ROOT
(24, 0.18%) о:SUFF
(24, 0.18%) один:ROOT
(24, 0.18%) ово:ROOT
(24, 0.18%) е:LINK
(24, 0.18%) генерал:ROOT
(24, 0.18%) а:SUFF
(10, 0.07%) дизель:ROOT
(5, 0.04%) флигель:ROOT
Всевозможные морфемы на концах наречий:
(195, 23.00%) о:END
(113, 13.33%) у:SUFF
(100, 11.79%) и:END
(87, 10.26%) ом:SUFF
(64, 7.55%) е:END
(64, 7.55%) а:END
(24, 2.83%) ую:END
(17, 2.00%) ю:END
(13, 1.53%) ому:END
(11, 1.30%) я:END
(10, 1.18%) ком:SUFF
(9, 1.06%) нибудь:POSTFIX
(8, 0.94%) ой:END
(8, 0.94%) ем:SUFF
(8, 0.94%) ее:END
(7, 0.83%) жды:SUFF
(7, 0.83%) либо:POSTFIX
(7, 0.83%) ок:SUFF
(6, 0.71%) о:SUFF
(5, 0.59%) ах:SUFF
(5, 0.59%) мя:SUFF
(5, 0.59%) ами:END
(5, 0.59%) ы:END
(3, 0.35%) ем:END
(3, 0.35%) а:SUFF
(3, 0.35%) ей:SUFF
(2, 0.24%) раз:ROOT
(2, 0.24%) ам:END
(2, 0.24%) сь:POSTFIX
(2, 0.24%) ова:SUFF
(2, 0.24%) ою:END
(2, 0.24%) ость:SUFF
(

In [ ]:
before_hyphen_words = []
for word in make_lemmas(hyphen_words):
    i = word.morpheme_indexes(MorphemeLabel.HYPH) [0] # позиция дефиса
    if word.morphemes[i-1].part_text != 'о' or word.morphemes[i-1].label != MorphemeLabel.LINK:
        before_hyphen_words.append(Word(word.morphemes[:]))

#before_hyphen_words.sort(key=lambda word: word.morphemes[-1])
for word in before_hyphen_words:
    print(word.get_word())

социал-предательского
масс-спектроскопический
национал-социалистский
дизель-электроходного
социал-соглашательский
редукционно-охладительный
социал-империалистический
стираный-перестираный
флигель-адъютантского
один-разъединственный
планово-организационный
социал-империалистский
социал-революционного
масс-спектрометрический
встречный-поперечного
социал-патриотический
дизель-электрического
летне-оздоровительный
генерал-губернаторский
штопаный-перештопаный
гамма-терапевтический
национал-демократический
социал-демократический
национал-либерального


### Статистика по одинаковым словоформам с разными частями речи и/или разными разборами

In [ ]:
    repeats = defaultdict(list) # Counter([word.get_word() for word in train_part]).most_common()
    for word in train_part:
        repeats[word.get_word()]. append((word.sp, str(word)))

In [ ]:
    for word in list(repeats):
        repeats[word] = sorted(set(repeats[word]))
        if len(repeats[word]) == 1: 
            repeats.pop(word)

In [ ]:
    words = set([word.get_word() for word in train_part])
    print(f"Кол-во словоформ, имеющих несколько различных частей речи и/или разборов слова: \
    {len(repeats)} ({len(repeats)/len(words)*100:.2f}% от всех различных слов)", end='\n\n')
    for word, pars in list(repeats.items())[:30]:
        print(word, *pars, sep='\n')

Кол-во словоформ, имеющих несколько различных частей речи и/или разборов слова: 15046 (1.34% от всех различных слов)

передняя
('ADJ', 'перед:ROOT/н:SUFF/яя:END')
('NOUN', 'перед:ROOT/н:SUFF/яя:END')
передней
('ADJ', 'перед:ROOT/н:SUFF/ей:END')
('NOUN', 'перед:ROOT/н:SUFF/ей:END')
переднюю
('ADJ', 'перед:ROOT/н:SUFF/юю:END')
('NOUN', 'перед:ROOT/н:SUFF/юю:END')
('VERB', 'пере:PREF/дн:ROOT/ю:SUFF/ю:END')
жила
('NOUN', 'жил:ROOT/а:END')
('VERB', 'жи:ROOT/л:SUFF/а:END')
жилой
('ADJ', 'жи:ROOT/л:SUFF/ой:END')
('NOUN', 'жил:ROOT/ой:END')
фрезерно-центровальный
('ADJ', 'фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ый:END')
('NOUN', 'фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ый:END')
фрезерно-центровального
('ADJ', 'фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ого:END')
('NOUN', 'фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ого:END')
фрезерно-центровальному
('ADJ', 'фрез:ROOT

In [ ]:
    diff_POS = Counter([tuple(zip(*l))[0] for l in repeats.values()])
    
    print(f"Кол-во различных вариантов нескольких частей речи для одной словоформы (всего: {len(diff_POS)}):", end='\n\n')
    summ = sum(diff_POS.values())
    for POS, count in diff_POS.most_common():
        print(f"{POS} - {count} ({count/summ*100:.2f}%)")

Кол-во различных вариантов нескольких частей речи для одной словоформы (всего: 29):

('ADJ', 'NOUN') - 9856 (65.51%)
('ADJ', 'NOUN', 'NOUN') - 1232 (8.19%)
('NOUN', 'NOUN') - 1141 (7.58%)
('ADJ', 'PART') - 1101 (7.32%)
('NOUN', 'VERB') - 754 (5.01%)
('VERB', 'VERB') - 543 (3.61%)
('GRND', 'NOUN') - 145 (0.96%)
('ADV', 'NOUN') - 99 (0.66%)
('ADJ', 'ADJ') - 55 (0.37%)
('ADJ', 'GRND') - 27 (0.18%)
('ADJ', 'ADJ', 'NOUN') - 20 (0.13%)
('ADJ', 'VERB') - 16 (0.11%)
('NOUN', 'VERB', 'VERB') - 7 (0.05%)
('ADV', 'GRND') - 7 (0.05%)
('GRND', 'VERB') - 7 (0.05%)
('ADJ', 'NOUN', 'PART') - 6 (0.04%)
('GRND', 'GRND') - 6 (0.04%)
('PART', 'PART') - 5 (0.03%)
('ADV', 'VERB') - 4 (0.03%)
('ADJ', 'NOUN', 'VERB') - 2 (0.01%)
('ADJ', 'ADJ', 'NOUN', 'NOUN') - 2 (0.01%)
('NOUN', 'NOUN', 'NOUN') - 2 (0.01%)
('NOUN', 'NOUN', 'VERB') - 2 (0.01%)
('ADJ', 'ADV') - 2 (0.01%)
('ADV', 'NOUN', 'VERB') - 1 (0.01%)
('ADJ', 'ADV', 'NOUN') - 1 (0.01%)
('ADJ', 'NOUN', 'NOUN', 'VERB') - 1 (0.01%)
('GRND', 'NOUN', 'NOUN') -

In [ ]:
def run_length_encoding(x):
    """
    Make run-length encoding.
    The first one contains numbers, and the second one - how many times they need to be repeated.
    x = [2, 2, 2, 3, 3, 3, 5, 2] -> [2, 3, 5, 2], [3, 3, 1, 1]
    """
    i, num, rep = 0, [], []
    while i < len(x):
        num.append(x[i])
        rep.append(0)
        while i < len(x) and x[i] == num[-1]:
            rep[-1] += 1
            i += 1
    return num, rep

In [ ]:
    same_POS_diff_parsing = defaultdict(list)
    for word in repeats:
        POS, parsing = list(zip(*repeats[word]))
        POS, rep = run_length_encoding(POS)
        for i, pos in enumerate(POS):
            if rep[i] != 1:
                j = sum(rep[:i])
                same_POS_diff_parsing[word]. append((pos, parsing[j : j+rep[i]]))

    print(f"Кол-во словоформ, имеющих несколько различных разборов слова для одной части речи: \
    {len(same_POS_diff_parsing)} ({len(same_POS_diff_parsing)/len(words)*100:.2f}% от всех различных слов)", end='\n\n')
    for word, pars in list(same_POS_diff_parsing.items())[:30]:
        print(word)
        for p in pars:
            print(p[0])
            print(*p[1], sep='\n')

Кол-во словоформ, имеющих несколько различных разборов слова для одной части речи: 3018 (0.27% от всех различных слов)

фрезерно-центровальным
NOUN
фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ы:END/м:END
фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ым:END
радиально-сверлильным
NOUN
ради:ROOT/альн:SUFF/о:LINK/-:HYPH/сверл:ROOT/и:SUFF/ль:SUFF/н:SUFF/ы:END/м:END
ради:ROOT/альн:SUFF/о:LINK/-:HYPH/сверл:ROOT/и:SUFF/ль:SUFF/н:SUFF/ым:END
светлейшим
NOUN
свет:ROOT/л:SUFF/ейш:SUFF/и:END/м:END
свет:ROOT/л:SUFF/ейш:SUFF/им:END
морильни
NOUN
мор:ROOT/и:SUFF/ль:SUFF/н:SUFF/и:END
мор:ROOT/и:SUFF/льн:SUFF/и:END
пампасским
NOUN
пампас:ROOT/ск:SUFF/и:END/м:END
пампас:ROOT/ск:SUFF/им:END
комплексно-механизированным
NOUN
комплекс:ROOT/н:SUFF/о:LINK/-:HYPH/механ:ROOT/из:SUFF/ир:SUFF/ова:SUFF/нн:SUFF/ы:END/м:END
комплекс:ROOT/н:SUFF/о:LINK/-:HYPH/механ:ROOT/из:SUFF/ир:SUFF/ова:SUFF/нн:SUFF/ым:END
полуголодным
NOUN
пол:ROOT/у:SUFF/голод:ROOT/н:SUFF

In [ ]:
    same_POS = Counter([tuple([l[0][0]])*len(l[0][1]) if len(l) == 1 else tuple(sorted(tuple([l[0][0]])*len(l[0][1]) + tuple([l[1][0]])*len(l[1][1])))
                        for l in same_POS_diff_parsing.values()])
    
    summ = sum(same_POS.values())
    print("Варианты одинаковых частей речи для одной словоформы с различными разборами слова:")
    for POS, count in same_POS.most_common():
        print(f"{POS} - {count} ({count/summ*100:.2f}%)")

Варианты одинаковых частей речи для одной словоформы с различными разборами слова:
('NOUN', 'NOUN') - 2377 (78.76%)
('VERB', 'VERB') - 550 (18.22%)
('ADJ', 'ADJ') - 75 (2.49%)
('GRND', 'GRND') - 6 (0.20%)
('PART', 'PART') - 5 (0.17%)
('ADJ', 'ADJ', 'NOUN', 'NOUN') - 2 (0.07%)
('NOUN', 'NOUN', 'NOUN') - 2 (0.07%)
('VERB', 'VERB', 'VERB') - 1 (0.03%)


In [ ]:
for word in same_POS_diff_parsing:
    if same_POS_diff_parsing[word][0][0] == 'PART':
        print(word)
        print(*same_POS_diff_parsing[word][0][1], sep='\n')

обобьющий
о:PREF/бобь:ROOT/ющ:SUFF/ий:END
обо:PREF/бь:ROOT/ющ:SUFF/ий:END
попрущий
по:PREF/пру:ROOT/щий:END
попр:ROOT/ущ:SUFF/ий:END
обобьющийся
о:PREF/бобь:ROOT/ющ:SUFF/ий:END/ся:POSTFIX
обо:PREF/бь:ROOT/ющ:SUFF/ий:END/ся:POSTFIX
охающий
о:PREF/ха:ROOT/ющ:SUFF/ий:END
ох:ROOT/а:SUFF/ющ:SUFF/ий:END
стающий
с:PREF/та:ROOT/ющ:SUFF/ий:END
ста:ROOT/ющ:SUFF/ий:END


In [ ]:
for word in same_POS_diff_parsing:
    if len(same_POS_diff_parsing[word][0][1]) >= 3:
        print(word, same_POS_diff_parsing[word])

шейки [('NOUN', ('ше:ROOT/йк:SUFF/и:END', 'шей:ROOT/к:SUFF/и:END', 'шейк:ROOT/и:END'))]
тельца [('NOUN', ('тел:ROOT/ьц:SUFF/а:END', 'тель:ROOT/ц:SUFF/а:END', 'тельц:ROOT/а:END'))]
смели [('VERB', ('с:PREF/ме:ROOT/л:SUFF/и:END', 'с:PREF/мел:ROOT/и:END', 'сме:ROOT/л:SUFF/и:END'))]


In [ ]:
for word in same_POS_diff_parsing:
    if len(same_POS_diff_parsing[word]) == 2:
        print(word, *same_POS_diff_parsing[word], sep='\n')

взводным
('ADJ', ('вз:PREF/вод:ROOT/н:SUFF/ым:END', 'взвод:ROOT/н:SUFF/ым:END'))
('NOUN', ('взвод:ROOT/н:SUFF/ы:END/м:END', 'взвод:ROOT/н:SUFF/ым:END'))
половой
('ADJ', ('пол:ROOT/ов:SUFF/ой:END', 'полов:ROOT/ой:END'))
('NOUN', ('пол:ROOT/ов:SUFF/ой:END', 'полов:ROOT/ой:END'))


In [ ]:
    diff_POS_same_parsing = defaultdict(list)
    for word in repeats:
        POS, parsing = sorted(zip(*repeats[word]), key=lambda x: x[1])
        parsing, rep = run_length_encoding(parsing)
        for i, pars in enumerate(parsing):
            if rep[i] != 1:
                j = sum(rep[:i])
                diff_POS_same_parsing[word]. append((pars, POS[j : j+rep[i]]))

    print(f"Кол-во словоформ, имеющих несколько различных частей речи для одноого разбора: \
    {len(diff_POS_same_parsing)} ({len(diff_POS_same_parsing)/len(words)*100:.2f}% от всех различных слов)", end='\n\n')
    print(*list(diff_POS_same_parsing.items())[:50], sep='\n')

Кол-во словоформ, имеющих несколько различных частей речи для одноого разбора: 7601 (0.68% от всех различных слов)

('передняя', [('перед:ROOT/н:SUFF/яя:END', ('ADJ', 'NOUN'))])
('передней', [('перед:ROOT/н:SUFF/ей:END', ('ADJ', 'NOUN'))])
('переднюю', [('перед:ROOT/н:SUFF/юю:END', ('ADJ', 'NOUN'))])
('фрезерно-центровальный', [('фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ый:END', ('ADJ', 'NOUN'))])
('фрезерно-центровального', [('фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ого:END', ('ADJ', 'NOUN'))])
('фрезерно-центровальному', [('фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ому:END', ('ADJ', 'NOUN'))])
('фрезерно-центровальном', [('фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ом:END', ('ADJ', 'NOUN'))])
('радиально-сверлильный', [('ради:ROOT/альн:SUFF/о:LINK/-:HYPH/сверл:ROOT/и:SUFF/ль:SUFF/н:SUFF/ый:END', ('ADJ', 'NOUN'))])
('радиально-сверлильного', [('ради:ROOT/ал

In [ ]:
    same_parsing = Counter([l[0][1] for l in diff_POS_same_parsing.values()])
    
    print("Варианты разных частей речи для одной словоформы с одинаковыми разборами слова:")
    summ = sum(same_parsing.values())
    for POS, count in same_parsing.most_common():
        print(f"{POS} - {count} ({count/summ*100:.2f}%)")

Варианты разных частей речи для одной словоформы с одинаковыми разборами слова:
('ADJ', 'NOUN') - 6014 (79.12%)
('ADJ', 'PART') - 1009 (13.27%)
('NOUN', 'VERB') - 484 (6.37%)
('GRND', 'NOUN') - 41 (0.54%)
('ADV', 'NOUN') - 37 (0.49%)
('GRND', 'VERB') - 7 (0.09%)
('ADJ', 'NOUN', 'PART') - 5 (0.07%)
('ADJ', 'VERB') - 3 (0.04%)
('ADV', 'VERB') - 1 (0.01%)


In [ ]:
    for word in diff_POS_same_parsing:
        if diff_POS_same_parsing[word][0][1] == ('ADJ', 'VERB') or diff_POS_same_parsing[word][0][1] == ('ADV', 'VERB'):
            print(word, diff_POS_same_parsing[word])

княжим [('княж:ROOT/им:END', ('ADJ', 'VERB'))]
горячим [('горяч:ROOT/им:END', ('ADJ', 'VERB'))]
синим [('син:ROOT/им:END', ('ADJ', 'VERB'))]
сродни [('с:PREF/родн:ROOT/и:END', ('ADV', 'VERB'))]


In [ ]:
    for word in diff_POS_same_parsing:
        if len(diff_POS_same_parsing[word][0][1]) == 3:
            print(word, diff_POS_same_parsing[word])

председательствующий [('председатель:ROOT/ств:SUFF/у:SUFF/ющ:SUFF/ий:END', ('ADJ', 'NOUN', 'PART'))]
замыкающий [('за:PREF/мык:ROOT/а:SUFF/ющ:SUFF/ий:END', ('ADJ', 'NOUN', 'PART'))]
командующий [('команд:ROOT/у:SUFF/ющ:SUFF/ий:END', ('ADJ', 'NOUN', 'PART'))]
успевающий [('успе:ROOT/ва:SUFF/ющ:SUFF/ий:END', ('ADJ', 'NOUN', 'PART'))]
разводящий [('раз:PREF/вод:ROOT/ящ:SUFF/ий:END', ('ADJ', 'NOUN', 'PART'))]


In [ ]:
    diff_POS_diff_parsing = defaultdict(list)
    for word in repeats:
        for i in range(len(repeats[word])-1):
            for j in range(i+1, len(repeats[word])):
                if repeats[word][i][0] != repeats[word][j][0] and repeats[word][i][1] != repeats[word][j][1]:
                    diff_POS_diff_parsing[word]. append(tuple(zip(repeats[word][i], repeats[word][j])))
    
    print(f"Кол-во словоформ, имеющих несколько различных разборов слова для различных частей речи соответственно: \
    {len(diff_POS_diff_parsing)} ({len(diff_POS_diff_parsing)/len(words)*100:.2f}% от всех различных слов)", end='\n\n')
    # print(*list(diff_POS_diff_parsing.items())[:50], sep='\n')
    for word, pars in list(diff_POS_diff_parsing.items())[:30]:
        print(word)
        for p in pars:
            print(*zip(*p), sep='\n')

Кол-во словоформ, имеющих несколько различных разборов слова для различных частей речи соответственно: 5702 (0.51% от всех различных слов)

переднюю
('ADJ', 'перед:ROOT/н:SUFF/юю:END')
('VERB', 'пере:PREF/дн:ROOT/ю:SUFF/ю:END')
('NOUN', 'перед:ROOT/н:SUFF/юю:END')
('VERB', 'пере:PREF/дн:ROOT/ю:SUFF/ю:END')
жила
('NOUN', 'жил:ROOT/а:END')
('VERB', 'жи:ROOT/л:SUFF/а:END')
жилой
('ADJ', 'жи:ROOT/л:SUFF/ой:END')
('NOUN', 'жил:ROOT/ой:END')
фрезерно-центровальным
('ADJ', 'фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ым:END')
('NOUN', 'фрез:ROOT/ер:SUFF/н:SUFF/о:LINK/-:HYPH/центр:ROOT/ова:SUFF/ль:SUFF/н:SUFF/ы:END/м:END')
радиально-сверлильным
('ADJ', 'ради:ROOT/альн:SUFF/о:LINK/-:HYPH/сверл:ROOT/и:SUFF/ль:SUFF/н:SUFF/ым:END')
('NOUN', 'ради:ROOT/альн:SUFF/о:LINK/-:HYPH/сверл:ROOT/и:SUFF/ль:SUFF/н:SUFF/ы:END/м:END')
пампасским
('ADJ', 'пампас:ROOT/ск:SUFF/им:END')
('NOUN', 'пампас:ROOT/ск:SUFF/и:END/м:END')
комплексно-механизированным
('ADJ', 'комплекс:ROOT/н:SUF

In [ ]:
    diff_parsing = Counter([tuple(zip(*l))[0] for l in diff_POS_diff_parsing.values()])
    
    print("Варианты разных частей речи для одной словоформы с различными разборами слова:")
    summ = sum(diff_parsing.values())
    for POS, count in diff_parsing.most_common():
        print(f"{POS} - {count} ({count/summ*100:.2f}%)")

Варианты разных частей речи для одной словоформы с различными разборами слова:
(('ADJ', 'NOUN'),) - 5093 (89.32%)
(('NOUN', 'VERB'),) - 276 (4.84%)
(('GRND', 'NOUN'),) - 106 (1.86%)
(('ADJ', 'PART'),) - 92 (1.61%)
(('ADV', 'NOUN'),) - 63 (1.10%)
(('ADJ', 'GRND'),) - 27 (0.47%)
(('ADJ', 'VERB'),) - 13 (0.23%)
(('ADJ', 'NOUN'), ('ADJ', 'NOUN')) - 10 (0.18%)
(('ADV', 'GRND'),) - 7 (0.12%)
(('NOUN', 'VERB'), ('NOUN', 'VERB')) - 3 (0.05%)
(('ADV', 'VERB'),) - 3 (0.05%)
(('ADJ', 'VERB'), ('NOUN', 'VERB')) - 2 (0.04%)
(('ADJ', 'ADV'),) - 2 (0.04%)
(('ADJ', 'NOUN'), ('ADJ', 'NOUN'), ('ADJ', 'NOUN')) - 1 (0.02%)
(('ADV', 'VERB'), ('NOUN', 'VERB')) - 1 (0.02%)
(('ADJ', 'ADV'), ('ADV', 'NOUN')) - 1 (0.02%)
(('ADJ', 'NOUN'), ('ADJ', 'VERB'), ('NOUN', 'VERB'), ('NOUN', 'VERB')) - 1 (0.02%)
(('ADJ', 'PART'), ('NOUN', 'PART')) - 1 (0.02%)


In [ ]:
    for word in diff_POS_diff_parsing:
        if diff_POS_diff_parsing[word][0][0] == ('ADJ', 'ADV'):
            print(word, *zip(*diff_POS_diff_parsing[word][0]), sep='\n')

броском
('ADJ', 'брос:ROOT/к:SUFF/ом:END')
('ADV', 'брос:ROOT/к:SUFF/ом:SUFF')
добром
('ADJ', 'добр:ROOT/ом:END')
('ADV', 'добр:ROOT/ом:SUFF')
стойком
('ADJ', 'стой:ROOT/к:SUFF/ом:END')
('ADV', 'стой:ROOT/ком:SUFF')


In [ ]:
    for word in diff_POS_diff_parsing:
        if len(diff_POS_diff_parsing[word]) >= 3:
            print(word, *diff_POS_diff_parsing[word], sep='\n')

взводным
(('ADJ', 'NOUN'), ('вз:PREF/вод:ROOT/н:SUFF/ым:END', 'взвод:ROOT/н:SUFF/ы:END/м:END'))
(('ADJ', 'NOUN'), ('вз:PREF/вод:ROOT/н:SUFF/ым:END', 'взвод:ROOT/н:SUFF/ым:END'))
(('ADJ', 'NOUN'), ('взвод:ROOT/н:SUFF/ым:END', 'взвод:ROOT/н:SUFF/ы:END/м:END'))
ловчим
(('ADJ', 'NOUN'), ('лов:ROOT/ч:SUFF/им:END', 'лов:ROOT/ч:SUFF/и:END/м:END'))
(('ADJ', 'VERB'), ('лов:ROOT/ч:SUFF/им:END', 'ловч:ROOT/им:END'))
(('NOUN', 'VERB'), ('лов:ROOT/ч:SUFF/и:END/м:END', 'ловч:ROOT/им:END'))
(('NOUN', 'VERB'), ('лов:ROOT/ч:SUFF/им:END', 'ловч:ROOT/им:END'))


In [ ]:
    for word in diff_POS_diff_parsing:
        if len(diff_POS_diff_parsing[word]) == 2:
            print(word)
            for pars in diff_POS_diff_parsing[word]:
                print(*zip(*pars), sep='\n', end='\n\n')

переднюю
('ADJ', 'перед:ROOT/н:SUFF/юю:END')
('VERB', 'пере:PREF/дн:ROOT/ю:SUFF/ю:END')

('NOUN', 'перед:ROOT/н:SUFF/юю:END')
('VERB', 'пере:PREF/дн:ROOT/ю:SUFF/ю:END')

порой
('ADV', 'пор:ROOT/ой:END')
('VERB', 'по:PREF/ро:ROOT/й:END')

('NOUN', 'пор:ROOT/ой:END')
('VERB', 'по:PREF/ро:ROOT/й:END')

повести
('NOUN', 'повест:ROOT/и:END')
('VERB', 'по:PREF/вес:ROOT/ти:END')

('NOUN', 'повест:ROOT/и:END')
('VERB', 'по:PREF/вест:ROOT/и:END')

броском
('ADJ', 'брос:ROOT/к:SUFF/ом:END')
('ADV', 'брос:ROOT/к:SUFF/ом:SUFF')

('ADV', 'брос:ROOT/к:SUFF/ом:SUFF')
('NOUN', 'брос:ROOT/к:SUFF/ом:END')

извести
('NOUN', 'извест:ROOT/и:END')
('VERB', 'из:PREF/вест:ROOT/и:END')

('NOUN', 'извест:ROOT/и:END')
('VERB', 'извес:ROOT/ти:END')

нападающий
('ADJ', 'напад:ROOT/а:SUFF/ющ:SUFF/ий:END')
('PART', 'на:PREF/пад:ROOT/а:SUFF/ющ:SUFF/ий:END')

('NOUN', 'напад:ROOT/а:SUFF/ющ:SUFF/ий:END')
('PART', 'на:PREF/пад:ROOT/а:SUFF/ющ:SUFF/ий:END')

половой
('ADJ', 'пол:ROOT/ов:SUFF/ой:END')
('NOUN', 'полов:ROOT/